# T1574.002 - DLL Side-Loading
Adversaries may execute their own malicious payloads by side-loading DLLs. Similar to [DLL Search Order Hijacking](https://attack.mitre.org/techniques/T1574/001), side-loading involves hijacking which DLL a program loads. But rather than just planting the DLL within the search order of a program then waiting for the victim application to be invoked, adversaries may directly side-load their payloads by planting then invoking a legitimate application that executes their payload(s).

Side-loading takes advantage of the DLL search order used by the loader by positioning both the victim application and malicious payload(s) alongside each other. Adversaries likely use side-loading as a means of masking actions they perform under a legitimate, trusted, and potentially elevated system or software process. Benign executables used to side-load payloads may not be flagged during delivery and/or execution. Adversary payloads may also be encrypted/packed or otherwise obfuscated until loaded into the memory of the trusted process.(Citation: FireEye DLL Side-Loading)

## Atomic Tests

### Atomic Test #1 - DLL Side-Loading using the Notepad++ GUP.exe binary
GUP is an open source signed binary used by Notepad++ for software updates, and is vulnerable to DLL Side-Loading, thus enabling the libcurl dll to be loaded.
Upon execution, calc.exe will be opened.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Gup.exe binary must exist on disk at specified location (#{gup_executable})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1574.002\bin\GUP.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1574.002\bin\GUP.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1574.002/bin/GUP.exe?raw=true" -OutFile "PathToAtomicsFolder\T1574.002\bin\GUP.exe"

```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\T1574.002\bin\GUP.exe"
```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 1

#### Cleanup: 
```cmd
taskkill /F /IM calculator.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - DLL Side-Loading using the dotnet startup hook environment variable
Utilizing the dotnet_startup_hooks environment variable, this method allows for registering a global method in an assembly that will be executed whenever a .net core application is started. This unlocks a whole range of scenarios, from injecting a profiler to tweaking a static context in a given environment. [blog post](https://medium.com/criteo-engineering/c-have-some-fun-with-net-core-startup-hooks-498b9ad001e1)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: .Net SDK must be installed

##### Check Prereq Commands:
```cmd
if (Test-Path "C:\Program Files\dotnet\dotnet.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
winget install Microsoft.DotNet.SDK.6 --accept-source-agreements --accept-package-agreements -h > $null
echo.

```
##### Description: preloader must exist

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1574.002\bin\preloader.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1574.002/bin/preloader?raw=true" -OutFile "PathToAtomicsFolder\T1574.002\bin\preloader.dll"

```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
set DOTNET_STARTUP_HOOKS="PathToAtomicsFolder\T1574.002\bin\preloader.dll"
dotnet -h > nul
echo.
```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 2

#### Cleanup: 
```cmd
taskkill /F /IM calculator.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1574.002 -TestNumbers 2 -Cleanup

## Detection
Monitor processes for unusual activity (e.g., a process that does not use the network begins to do so) as well as the introduction of new files/programs. Track DLL metadata, such as a hash, and compare DLLs that are loaded at process execution time against previous executions to detect differences that do not correlate with patching or updates.